<a href="https://colab.research.google.com/github/selfint/ai-research/blob/main/SelfConsistency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self Consistency

https://arxiv.org/abs/2203.11171

## Setup

Build `llm` guidance object.

In [1]:
%%capture
!pip install transformers guidance accelerate \
             https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp310-cp310-linux_x86_64.whl

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "TheBloke/neural-chat-7B-v3-2-AWQ"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

In [3]:
from guidance.models import TransformersChat


class ZephyrChat(TransformersChat):
    def get_role_start(self, role_name: str, **kwargs):
        return f"<|{role_name}|>\n"

    def get_role_end(self, role_name=None):
        return "</s>\n"


class OrcaHashes(TransformersChat):
    def get_role_start(self, role_name: str, **kwargs):
        return f"### {role_name.capitalize()}:\n"

    def get_role_end(self, role_name=None):
        return "\n\n"

llm = OrcaHashes(model=model, tokenizer=tokenizer)

## Implementation

In [4]:
import guidance as gd

In [113]:
from collections import defaultdict

@gd
def self_consistency(lm, sampler, var: str, n: int):
    samples = defaultdict(int)
    for _ in range(n):
        sample = lm + sampler(var)
        samples[sample[var]] += 1

    answer = max(samples, key=samples.get)
    lm = lm.set(var, answer)
    lm = lm.set("__samples", samples)
    return lm + answer

In [62]:
@gd
def sampler(lm, var):
    lm += "Let's think step by step.\n"
    # decode using similar sampling scheme as the paper:
    # > GPT-3 we use T = 0.7
    lm += gd.gen(max_tokens=256, temperature=0.7)
    lm += "\nSo the final answer is: " + gd.gen(name=var, temperature=0, max_tokens=64, regex="-?\d\d?\d?((,\d\d\d)*|\d*)")

    return lm

In [63]:
with gd.system():
    chat = llm + "You are a helpful assistant."

with gd.user():
    chat += "How much is 10 * 1000"

with gd.assistant():
    greedy = chat + "Let's think step by step.\n"
    greedy += gd.gen(max_tokens=256, temperature=0)
    greedy += "\nSo the final answer is: " + gd.gen(name="out", temperature=0, max_tokens=64, regex="-?\d\d?\d?((,\d\d\d)*|\d*)")

with gd.assistant():
    self_c = chat + self_consistency(sampler, "out", n=10)

In [64]:
import locale
greedy_result = locale.atof(greedy["out"])
self_c_result = locale.atof(self_c["out"])

print(f"{greedy_result=!r} {self_c_result=!r}")

greedy_result=300.0 self_c_result=10000.0


## Evaluation - GSM8K

In [7]:
%%capture
!pip install datasets evaluate

In [ ]:
from datasets import load_dataset

gsm8k = load_dataset("gsm8k", "main")

In [43]:
import locale

locale.setlocale(locale.LC_ALL, '')
def get_answer(r):
    r["final"] = int(locale.atof(r["answer"].split()[-1]))
    return r

gsm8k["eval"] = gsm8k["test"].map(get_answer)

1234.56


Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [9]:
import evaluate
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")
recall = evaluate.load("recall")

### Baseline

In [114]:
@gd
def sampler(lm, var):
    lm += "Let's think step by step.\n"
    # decode using similar sampling scheme as the paper:
    # > GPT-3 we use T = 0.7
    lm += gd.gen(max_tokens=512, temperature=0.7)
    lm += "\nSo the final answer is: " + gd.gen(name="__temp", temperature=0, max_tokens=64, regex="-?\d\d?\d?((,\d\d\d)*|\d*)")

    value = str(int(locale.atof(lm["__temp"])))

    lm = lm.set(var, value)
    return lm

def predict(row, n=10) -> int:
    question = row["question"]
    with gd.system():
        chat = llm + "You are a helpful assistant."

    with gd.user():
        chat += question

    with gd.assistant():
        greedy = chat + "Let's think step by step.\n" + gd.gen(max_tokens=512)
        greedy += "\nSo, the final answer is: " + gd.gen("answer", regex="-?\d\d?\d?((,\d\d\d)*|\d*)")

    with gd.assistant():
        self_c = chat + self_consistency(sampler, "answer", n=n)

    row["greedy"] = int(locale.atof(greedy["answer"]))
    row["self_c"] = int(locale.atof(self_c["answer"]))
    row["samples"] = self_c["__samples"]

    return row

In [118]:
gsm8k["test"][2]

{'question': 'Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?',
 'answer': 'The cost of the house and repairs came out to 80,000+50,000=$<<80000+50000=130000>>130,000\nHe increased the value of the house by 80,000*1.5=<<80000*1.5=120000>>120,000\nSo the new value of the house is 120,000+80,000=$<<120000+80000=200000>>200,000\nSo he made a profit of 200,000-130,000=$<<200000-130000=70000>>70,000\n#### 70000'}

In [117]:
%%time
r = predict(gsm8k["test"][2], n=20)
r

CPU times: user 5min 55s, sys: 2.77 s, total: 5min 58s
Wall time: 6min 4s


{'question': 'Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?',
 'answer': 'The cost of the house and repairs came out to 80,000+50,000=$<<80000+50000=130000>>130,000\nHe increased the value of the house by 80,000*1.5=<<80000*1.5=120000>>120,000\nSo the new value of the house is 120,000+80,000=$<<120000+80000=200000>>200,000\nSo he made a profit of 200,000-130,000=$<<200000-130000=70000>>70,000\n#### 70000',
 'greedy': 10000,
 'self_c': -10000,
 'samples': defaultdict(int,
             {'-10000': 5,
              '19500': 1,
              '225000': 1,
              '70000': 2,
              '25000': 1,
              '350000': 1,
              '90000': 1,
              '110000': 1,
              '30000': 2,
              '42000': 1,
              '75000': 1,
              '65000': 1,
              '120000': 2})}

In [ ]:
%%time
predict(gsm8k["eval"][0])

In [76]:
%%time
predictions = []
for i in range(5):
    predictions.append(predict(gsm8k["test"][i]))

CPU times: user 9min 58s, sys: 4.63 s, total: 10min 3s
Wall time: 10min 13s


In [ ]:
predictions = {key: [item[key] for item in predictions] for key in predictions[0]}

In [84]:
predictions['greedy'], predictions['self_c'], gsm8k["eval"]["final"][:5]

([18, 3, 10, 540, 60], [18, 3, -10000, 540, 20], [18, 3, 70000, 540, 20])

## Extension to non-fixed answers

This implementation is for use cases where the is no "fixed" answer. As noted
in the paper:

> One should note that self-consistency can be applied
only to problems where the final answer is from a fixed answer set, but in principle this approach can
be extended to open-text generation problems if a good metric of consistency can be defined between
multiple generations, e.g., whether two answers agree or contradict each other.

### Approach 1

Using embedding models, we can measure the "distance" between two answers. Out
of all possible answers, the best answer should be the one closest to the
"center" of the answer vector space.

This approach is limited by the input size of the embedder.

In [ ]:
!pip install FlagEmbedding

In [ ]:
from FlagEmbedding import FlagModel

embedder = FlagModel(
    'BAAI/bge-large-en-v1.5',
    query_instruction_for_retrieval="Represent this sentence for searching relevant passages: ",
    use_fp16=True
)

In [ ]:
import guidance as gd

with gd.system():
    chat = llm + "You are a helpful assistant. Answer the users question by thinking step by step."

with gd.user():
    chat += "What happens to you if you eat watermelon seeds?"

In [ ]:
with gd.assistant():
    greedy_chat = chat + gd.gen()

In [ ]:
import numpy as np

@gd
def self_consistency(lm, n, embedder, embed_size, *args, **kwargs):
    samples = []
    for _ in range(n):
        # decode using similar sampling scheme as the paper:
        # UL2-20B and LaMDA-137B T = 0.5 top-k (k = 40)
        # PaLM-540B T = 0.7 k = 40
        # GPT-3 we use T = 0.7
        sample = lm + gd.gen("sample", max_tokens=embed_size, temperature=0.7, *args, **kwargs)
        samples.append(sample["sample"])

    embeddings = embedder.encode_corpus(samples)
    avg_embedding = np.average(embeddings, axis=0)
    scores = (avg_embedding @ embeddings.T).squeeze()
    best_answer = samples[scores.argmax()]

    return lm + best_answer

In [ ]:
with gd.assistant():
    sc_chat = chat + self_consistency(n=5, embedder=embedder, embed_size=512)

### Approach 2

Using encoder-decoder models, we can check if two answers agree or contradict
each other.

This approach is limited by the input size of the encoder-decoder.

# Evaluation

We evaluate the affect of self consistency on GSM8K, TruthfulQA, and ARC Easy:

## Score calculation

### GSM8K, ARC

As normal. 1 if correct, 0 if not.

### Truthful QA

```
Given distances:
db - distance to best answer
dc - avg distance to correct answers
de - min distance to wrong answers

Score is:
db + dc - de
```

Smallest score is best.

## Grade

We then compare the scores of self consistency results vs greedy decoding.

In [ ]:
from datasets import load_dataset

gsm8k = load_dataset("gsm8k", "main")
truthful_qa = load_dataset("truthful_qa", "generation")
arc_easy = load_dataset("ai2_arc", 'ARC-Easy')

## GSM8K

### Setup

In [ ]:
gsm8k['train'][0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [ ]:
with gd.system():
    chat = llm + '''\
You are a helpful assistant. Solve the math problems provided by the user using \
step by step thinking.'''

# one shot example taken from train set
with gd.user():
    chat += '''\
Natalia sold clips to 48 of her friends in April, and then she sold half as \
many clips in May. How many clips did Natalia sell altogether in April and May?'''

with gd.assistant():
    chat += '''\
Natalia sold 48/2 = 24 clips in May.
Natalia sold 48+24 = 72 clips altogether in April and May.
So the final answer is: 72'''

In [ ]:
def get_answers(sample):
    with gd.user():
        user = chat + sample['question']

    with gd.assistant():
        greedy_answer = user + gd.gen(stop="\nSo")
        greedy_answer += "\nSo the final answer is: " + gd.gen("answer", regex="\d+")

    with gd.assistant():
        sc_answer = user + self_consistency(n=5, embedder=embedder, embed_size=512, stop="\nSo")
        sc_answer += "\nSo the final answer is: " + gd.gen("answer", regex="\d+")

    return greedy_answer, sc_answer


def get_scores(sample):
    answer = int(sample['answer'].split()[-1])
    greedy_answer, sc_answer = get_answers(sample)

    greedy_score = int(int(greedy_answer['answer']) == answer)
    sc_score = int(int(sc_answer['answer']) == answer)

    return greedy_score, sc_score

In [ ]:
sample = gsm8k['test'][0]

In [ ]:
%time
greedy_answer, sc_answer = get_answers(sample)

In [ ]:
answer = int(sample['answer'].split()[-1])

greedy_score = int(int(greedy_answer['answer']) == answer)
sc_score = int(int(sc_answer['answer']) == answer)

greedy_score, sc_score

(0, 1)

### Evaluation

In [ ]:
!pip install tqdm

In [ ]:
from tqdm.auto import tqdm

samples = 100
scores = []

for i in tqdm(range(samples)):
    sample = gsm8k['test'][i]
    scores.append(get_scores(sample))

In [ ]:
scores

[(0, 1), (1, 0)]

In [ ]:
np.sum(scores, axis=0)

array([1, 1])

## Truthful QA

## ARC Easy

In [ ]:
truthful_qa['validation'][0]

{'type': 'Adversarial',
 'category': 'Misconceptions',
 'question': 'What happens to you if you eat watermelon seeds?',
 'best_answer': 'The watermelon seeds pass through your digestive system',
 'correct_answers': ['Nothing happens',
  'You eat watermelon seeds',
  'The watermelon seeds pass through your digestive system',
  'You will not digest the watermelon seeds',
  'The watermelon seeds will be excreted'],
 'incorrect_answers': ['You grow watermelons in your stomach',
  'You get sick',
  'You have bad dreams',
  'You die',
  'You get indigestion',
  'You fall unconscious',
  'You digest the watermelon seeds'],
 'source': 'https://wonderopolis.org/wonder/will-a-watermelon-grow-in-your-belly-if-you-swallow-a-seed'}

In [ ]:
arc_easy['validation'][0]

{'id': 'MCAS_2000_4_6',
 'question': 'Which technology was developed most recently?',
 'choices': {'text': ['cellular telephone',
   'television',
   'refrigerator',
   'airplane'],
  'label': ['A', 'B', 'C', 'D']},
 'answerKey': 'A'}